In [ ]:
# default_exp sample_code_test

In [2]:
#export
import time

def some_function_to_run_on_cloud():
    print('running sample_code_test.some_function_to_run_on_cloud')
    print('in main before sleep 1')
    time.sleep(2)
    print('in main after sleep 2')
    time.sleep(5)
    print('in main after sleep 3')
    time.sleep(5)
    print('in main after sleep 4')

In [ ]:
#hide
some_function_to_run_on_cloud()

In [ ]:
!python3 -m gcp_runner.entry_point --module-name='gcp_runner.sample_code' --function-name='some_function_to_run_on_cloud'

In [ ]:
!docker run -v `pwd`/gcp_runner:/gcp_runner 'gcr.io/deeplearning-platform-release/tf2-cpu.2-1' python3 -u -m gcp_runner.entry_point --module-name='gcp_runner.sample_code' --function-name='some_function_to_run_on_cloud'

In [ ]:
!gcloud ai-platform local train --job-dir='job-dir' --module-name=gcp_runner.entry_point \
    --package-path=`pwd`/gcp_runner -- \
    --module-name='gcp_runner.sample_code' --function-name='some_function_to_run_on_cloud'    

In [ ]:
!ls

In [ ]:
!docker build -f Dockerfile -t gcr.io/alekseyv-scalableai-dev/tf2-cpu.2-1 ./

In [ ]:
#hide
from gcp_runner.core import export_and_reload_all
export_and_reload_all()

gcp_runner.run_local.run_python(some_function_to_run_on_cloud)

In [ ]:
#hide
import time

import gcp_runner
from gcp_runner.core import export_and_reload_all
#export_and_reload_all(silent=True, ignore_errors=False)
export_and_reload_all()

#gcp_runner.run_local.run_python(some_function_to_run_on_cloud)

# gcp_runner.run_local.run_docker(
#     some_function_to_run_on_cloud, 
#     'gcr.io/deeplearning-platform-release/tf2-cpu.2-1')

gcp_runner.local_runner.run_on_ai_platform(
    some_function_to_run_on_cloud, 
    'job-dir')

# need to put package to docker container
# gcp_runner.ai_platform_runner.run_docker_image(
#     some_function_to_run_on_cloud,
#     'gs://alekseyv-scalableai-dev-criteo-model-bucket/test-job-dir',
#     build_docker_file='Dockerfile',
#     master_image_uri='gcr.io/alekseyv-scalableai-dev/tf2-cpu.2-1')

# gcp_runner.ai_platform_runner.run_package(
#     some_function_to_run_on_cloud,
#     'gs://alekseyv-scalableai-dev-criteo-model-bucket/test-job-dir')

#build_docker_file='Dockerfile',


# gcp_runner.kubernetes_runner.run_docker_image(
#     some_function_to_run_on_cloud,
#     'gs://alekseyv-scalableai-dev-criteo-model-bucket/test-job-dir',
#     image_uri='gcr.io/alekseyv-scalableai-dev/tf2-cpu.2-1')

Converted 00_core.ipynb.
Converted 01_ai_platform_constants.ipynb.
Converted ai_platform_runner.ipynb.
Converted entry_point.ipynb.
Converted index.ipynb.
Converted kubernetes_runner.ipynb.
Converted local_runner.ipynb.
Converted sample_code_test.ipynb.
Running training job using local Cloud AI:
gcloud ai-platform local train \ 
 --job-dir=job-dir \ 
 --module-name=gcp_runner.entry_point \ 
 --package-path=/usr/local/google/home/alekseyv/vlasenkoalexey/gcp_runner/gcp_runner  \ 
 -- \ 
 --job-dir=job-dir \ 
 --module-name=gcp_runner.sample_code_test \ 
 --function-name=some_function_to_run_on_cloud


In [ ]:
from nbdev.export import *
notebook2script()


Converted 00_core.ipynb.
Converted 01_ai_platform_constants.ipynb.
Converted ai_platform_runner.ipynb.
Converted entry_point.ipynb.
Converted index.ipynb.
Converted kubernetes_runner.ipynb.
Converted local_runner.ipynb.
Converted sample_code_test.ipynb.


In [4]:
from gcp_runner.core import export_and_reload_all
export_and_reload_all(silent=True, ignore_errors=False)

In [ ]:
import gcp_runner.ai_platform_runner

gcp_runner.ai_platform_runner.run_package(
     some_function_to_run_on_cloud, 
     'gs://alekseyv-scalableai-dev-criteo-model-bucket/test-job-dir')

In [ ]:
import gcp_runner.kubernetes_runner
from gcp_runner import ai_platform_constants

gcp_runner.kubernetes_runner.run_docker_image(
    some_function_to_run_on_cloud,
    'gs://alekseyv-scalableai-dev-criteo-model-bucket/test-job-dir',
    image_uri='gcr.io/alekseyv-scalableai-dev/tf2-cpu.2-1',
    distribution_strategy_type=ai_platform_constants.DistributionStrategyType.MIRRORED_STRATEGY,
    use_distribution_strategy_scope=True,
    dry_run=True)

To see job output in tensorboard, run following command:
tensorboard --logdir=%s gs://alekseyv-scalableai-dev-criteo-model-bucket/test-job-dir
Generating kubernetes YAML file:
kind: Service
apiVersion: v1
metadata:
  name: kubernetes-runner-train-docker-chief-0
spec:
  type: ClusterIP
  clusterIP: None
  selector:
    job: kubernetes-runner-train-docker-chief-0
  ports:
  - port: 5000
---
kind: Pod
apiVersion: v1
metadata:
  name: kubernetes-runner-train-docker-chief-0
  labels:
    job: kubernetes-runner-train-docker-chief-0
  annotations:
    tf-version.cloud-tpus.google.com: "2.1"
spec:
  hostNetwork: true
  dnsPolicy: ClusterFirstWithHostNet
  restartPolicy: Never
  containers:
  - name: gcp-runner-image
    image: gcr.io/alekseyv-scalableai-dev/tf2-cpu.2-1
    imagePullPolicy: Always
    resources:
      limits:
        cpu: 3
        memory: 12G
        nvidia.com/gpu: 2

    env:
    - name: TF_CONFIG
      value: "{
  \"cluster\": {\"chief\": [\"kubernetes-runner-train-docker-c